# 研究有些股票开始上涨的原因.
本来想用 祥和实业 从20200728 开始的这波涨势为例子进行研究, 但是考虑到 daily data 的原因.是否可以找到一个从 20200810 开始上涨的 股票.
### 载入数据

In [1]:
import math
import datetime
import multiprocessing as mp
import tushare as ts
import os
import numpy as np
import pandas as pd
import datetime
p = print
# tushare 的初始化
token = '42e0612206685cd9bcd250cc5a3ef3931149303bc1d512c261174b5d'
pro = ts.pro_api(token)
# 获取当前上市公司列表，包括股票代码，注册地，行业，上市时间等数据
basic = pro.stock_basic(list_status = 'L')

In [2]:
# 检索所有股票, 找到一只,从 20200810 开始一直在上涨的股票.abs
upstart_time = '20200810'
select_stocklist = dict()
for one_tscode in basic['ts_code']:
    stock_data_p = ts.pro_bar(api = pro, ts_code = one_tscode, start_date = upstart_time, adj = 'qfq')
    #p(stock_data_p)
    # 因为从 20200810 到现在一共只有 10 个交易日, 则我们用下面这个判断规则来筛选:
    # 1, 上涨的天数,大于 8 ?
    if sum(stock_data_p['pct_chg'] > 0) > 8:
        if sum(stock_data_p['pct_chg'] > 3) > 5:
            select_stocklist[stock_data_p.iloc[0]['ts_code']] = stock_data_p


In [3]:
select_stocklist.keys()
# 002919 感觉上是符合要求的.

dict_keys(['002108.SZ', '002335.SZ', '002716.SZ', '002919.SZ', '002991.SZ', '300094.SZ', '300312.SZ', '300313.SZ', '300647.SZ', '300778.SZ', '300859.SZ', '600158.SH', '600226.SH', '600568.SH', '601456.SH', '603076.SH', '603167.SH', '603303.SH'])

In [4]:
# 用程序筛选:
# 多天盘整, 然后向上突破均线后形成上涨趋势.
# 观察 300204 的数据: 从20200619 到 20200701 是一个调整期,
# 而从 20200703 开始, 突破了均线后, 形成了一个上涨势头.
study_starttime = '20200601'
oneselect_stock = basic[basic['symbol'] == '300204']['ts_code'].iloc[0]
oneselect_stockdata = ts.pro_bar(api = pro, ts_code = oneselect_stock, start_date = study_starttime, adj = 'qfq')

In [5]:
# 要把数据的时间格式改一下.
oneselect_stockdata['trans_time'] = pd.to_datetime(oneselect_stockdata['trade_date'])
#print(oneselect_stockdata)
# 设定一个 检查的时间长度 len_Maverage
len_Maverage = 4
len_select = int(len_Maverage * 1.5)
# 找一段 ,在这一段里找跌幅绝大多数在 2% 以内, 偶尔有两个会在 3%左右.这个限制用 in2_limit 表示
in2_limit = 0.7
in3_limit = 0.95
###############################################################################
# 选择一个超过这个时间长度的调整期
for i_len in range(len_select + 1, len(oneselect_stockdata)):
    analysis_data = oneselect_stockdata.iloc[-i_len:-(i_len- len_select), :]
    up2_now = sum(abs(analysis_data['pct_chg']) <= 2)/len(analysis_data)
    if up2_now > in2_limit :
        up3_now = sum(abs(analysis_data['pct_chg']) <= 3)/len(analysis_data)
        if up3_now > in3_limit :
            close_beyond_open = []
            for i in range(len(analysis_data)):
                temp_data = (analysis_data.iloc[i,2] - analysis_data.iloc[i,5])/analysis_data.iloc[i,5]
                close_beyond_open.append(round(temp_data,5))
            up5_sum = sum(abs(np.array(close_beyond_open)) > 0.05)
            if up5_sum == 0 :
                # 到这里为止,已经可以从数据例子中: 观察 300204 的数据: 从20200619 到 20200701 是一个调整期,,在这个例子中准确额找到 20200619 这个日期,
                # 那么... 下一步是 ,在之后的一天 向上突破均线.(目前设定为三天内)



                break
p(analysis_data)
p('~~~~~~~~~~~~~~~~~~~')
p(i_len)



# 开盘收盘很接近


      ts_code trade_date   open   high    low  close  pre_close  change  \
40  300204.SZ   20200630  11.09  11.24  11.03  11.16      11.17   -0.01   
41  300204.SZ   20200629  11.31  11.56  11.13  11.17      10.98    0.19   
42  300204.SZ   20200624  11.16  11.17  10.88  10.98      11.16   -0.18   
43  300204.SZ   20200623  11.30  11.42  11.08  11.16      11.24   -0.08   
44  300204.SZ   20200622  11.24  11.36  11.14  11.24      11.24    0.00   
45  300204.SZ   20200619  11.37  11.47  11.23  11.24      11.54   -0.30   

    pct_chg       vol     amount trans_time  
40  -0.0895  31498.10  35105.126 2020-06-30  
41   1.7304  50637.52  57334.382 2020-06-29  
42  -1.6129  24718.68  27203.880 2020-06-24  
43  -0.7117  27886.50  31230.235 2020-06-23  
44   0.0000  26644.82  29972.600 2020-06-22  
45  -2.5997  41355.75  46785.171 2020-06-19  
~~~~~~~~~~~~~~~~~~~
20


In [9]:
    # 这里选出来的时间段的最后一天是: 20200630 , 逐一检验后续日期中是否有可以超过均线的天数.
# 检验的日期长度设定为: test_len
test_len = 3 # 检验后续三天的

#######################
import arrow
last_date = analysis_data.iloc[-1]['trade_date']
last_date_arrow = arrow.get(last_date, 'YYYYMMDD')
print(last_date_arrow)

# 获得 average line 的起始时间: 留了足够的时间长度, 获得 50 日平均线应该也是够了.
startdate_averline = last_date_arrow.shift(days = -30 * 2)
str_startdate_averline = startdate_averline.format('YYYYMMDD')
print(str_startdate_averline)

# 获得平均线数据:
# 均线, 取得上面数据的均线:
# vol : 成交量; amount : 成交额; ma_v_5 : 是五日平均的成交量.
# 这个五日平均线是自动计算,所以要获得 0701 的平均线结果 要提前足够的天数.
average_line = ts.pro_bar(api = pro,ts_code = oneselect_stock, start_date = str_startdate_averline,  ma=[5, 10,20])

average_line


2020-06-19T00:00:00+00:00
20200420


,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,ma5,ma_v_5,ma10,ma_v_10,ma20,ma_v_20
0,300204.SZ,20200825,11.98,12.28,11.86,12.19,11.90,0.29,2.4370,60426.16,73294.031,11.924,37678.736,11.941,40620.006,12.3310,77916.1805
1,300204.SZ,20200824,11.99,12.06,11.70,11.90,11.98,-0.08,-0.6678,29318.97,34785.237,11.896,32332.944,11.920,40914.710,12.3210,76741.3965
2,300204.SZ,20200821,11.80,12.04,11.73,11.98,11.79,0.19,1.6115,33366.14,39715.798,11.928,36215.746,11.974,42994.957,12.3255,77162.3900
3,300204.SZ,20200820,11.76,11.93,11.60,11.79,11.76,0.03,0.2551,25261.57,29804.960,11.934,39739.976,12.013,47592.631,12.3230,79949.1785
4,300204.SZ,20200819,11.96,12.01,11.72,11.76,12.05,-0.29,-2.4066,40020.84,47324.058,11.950,41600.232,12.082,53624.042,12.3615,84337.6970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,300204.SZ,20200424,11.23,11.45,10.76,10.83,11.32,-0.49,-4.3286,43260.22,47518.927,11.298,31679.432,NaN,NaN,NaN,NaN
83,300204.SZ,20200423,11.52,11.86,11.31,11.32,11.41,-0.09,-0.7888,33368.80,38564.828,NaN,NaN,NaN,NaN,NaN,NaN
84,300204.SZ,20200422,11.34,11.49,11.10,11.41,11.38,0.03,0.2636,28786.97,32449.520,NaN,NaN,NaN,NaN,NaN,NaN
85,300204.SZ,20200421,11.42,11.50,11.23,11.38,11.55,-0.17,-1.4719,27025.84,30736.488,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
enddate = oneselect_stockdata[oneselect_stockdata['trade_date'] == analysis_data.iloc[0]['trade_date']]
for i in range(test_len):
    test_date = oneselect_stockdata.iloc[enddate.index[0]-1-i]
    if i == 2:
        break

In [11]:
one_average_line = average_line[average_line['trade_date'] == test_date['trade_date']].iloc[0]
# 在这套数据里,其实还可以比较成交量的关系.以后作为一个研究方向.
# 使用的均线也可以作为一个研究的目标. 目前只选用了 5, 10 ,20 三个均线.
# 这里的判定条件可以有三种: 目前先用第一个最宽的标准
criterion_para = 'criterion_1'
# 1, 只要最高价超过均线就算超过: criterion_1
if criterion_para == 'criterion_1':
    if one_average_line['ma5'] <= one_average_line['high'] and \
        one_average_line['ma10'] <= one_average_line['high'] and \
            one_average_line['ma20'] <= one_average_line['high'] :
        print('i get right one using criterion 1')
# 2, 必须开盘价超过均线才算超过: criterion_2
if criterion_para == 'criterion_2':
    if one_average_line['ma5'] <= one_average_line['open'] and \
        one_average_line['ma10'] <= one_average_line['open'] and \
            one_average_line['ma20'] <= one_average_line['open'] :
        print('i get right one using criterion 2')
# 3, 必须收盘价超过均线才算超过: criterion_3
if criterion_para == 'criterion_3':
    if one_average_line['ma5'] <= one_average_line['close'] and \
        one_average_line['ma10'] <= one_average_line['close'] and \
            one_average_line['ma20'] <= one_average_line['close'] :
        print('i get right one using criterion 3')

i get right one using criterion 1


### 好的, 以上做出了一个选择标准, 后面来设定上涨标准:

In [12]:
# test_date 是我目前获得, 并通过判断的数据. 这时候我们可以去判断后面是否产生了上涨.
# 一个简单粗暴的判定方法就是: 后几天的 pct_chg 的求和 是否大于某一个数字.




In [14]:
test_date

ts_code                 300204.SZ
trade_date               20200703
open                        11.29
high                        11.34
low                            11
close                       11.18
pre_close                   11.09
change                       0.09
pct_chg                    0.8115
vol                         51172
amount                    57098.4
trans_time    2020-07-03 00:00:00
Name: 37, dtype: object